# (2-10) Sequential Recommendation 실습 (상)

## 실습 개요

1) 실습 목적

GoodReads review 데이터셋을 활용하여 대표적인 Sequential Recommendation System인 SASRec 모델을 구현해 보겠습니다. GoodReads review 데이터는 user id, rating, book id, review, timestamp 등 다양한 column들을 포함하는데, 이번 실습에서는 user id, book id, timestamp column만을 활용하여 sequential recommendation에 필요한 user sequence 데이터를 만들어 봅니다.
실제 사용자 데이터에 sequential recommendation 모델을 적용해 보는 경험을 통해 실제 추천시스템 환경에서 사용자 맞춤형 추천이 어떻게 이루어지는지 경험해보실 수 있습니다.

2) 수강 목표

- 목표 1: GoodReads 리뷰 데이터셋의 user id, book id, timestamp 컬럼을 이용하여 사용자 시퀀스 데이터를 만드는 과정을 경험하고, 데이터 전처리와 시퀀스 데이터 구성의 기본 원칙을 배웁니다.
- 목표 2: SASRec 모델을 PyTorch로 직접 경험해 보는 경험을 통해 Sequential Recommendation System의 구현 방법을 실습합니다.
- 목표 3: SASRec 모델의 아키텍처와 작동 원리를 깊이 있게 이해합니다.
- 목표 4: 실제 사용자 데이터에 SASRec 모델을 적용해보며, 사용자 개인화 추천이 실제 추천 시스템 환경에서 어떻게 이루어지는지 경험합니다.


## 실습 목차

* 1. Exploring Data
* 2. Data Preprocessing
* 3. SASRec Model
  * 3-1. Batch Sampling
  * 3-2. 모델 구현
  * 3-3. Model Training
  * 3-4. Model Evaluation
* 4. 마치며

###  데이터셋 개요

* 데이터셋: GoodReads
* 데이터셋 개요: GoodReads 데이터셋은 사용자 리뷰와 함께 수많은 책에 대한 상세 정보를 포함하고 있어, 추천 알고리즘의 개발 및 검증에 사용됩니다. 주로 사용자별로 다양한 책에 대한 평점과 리뷰 정보를 제공합니다.
* 데이터셋 저작권: GoodReads 데이터셋은 주로 교육 및 연구 목적으로 사용되며, 이용에 관한 자세한 저작권 정보는 GoodReads의 공식 웹사이트에서 확인할 수 있습니다.



### 환경 설정
- 패키지 설치 및 임포트
```
import requests
import gzip
import json
import re
import os
import sys
import pandas as pd
from collections import defaultdict
import math
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
```


In [ ]:
# 라이브러리 임포트
import gdown
import requests
import gzip
import json
import re
import os
import sys
import pandas as pd
from collections import defaultdict
import math
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F

## 1. Exploring Data
```
💡 목차 개요 : GoodReads review 데이터셋을 다운로드하고, Pandas 라이브러리를 활용하여 데이터셋을 살펴봅니다.
```

먼저 GoodReads review dataset을 다운로드합니다.

In [ ]:
file_id = "1JO1Y3McBAPQuXHG1tezbk1n7_MnegA_1"
output = "./goodreads_reviews_spoiler.json.gz" # 저장 위치 및 저장할 파일 이름
gdown.download(id=file_id, output=output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1JO1Y3McBAPQuXHG1tezbk1n7_MnegA_1
To: /content/goodreads_reviews_spoiler.json.gz
100%|██████████| 620M/620M [00:11<00:00, 53.1MB/s]


'./goodreads_reviews_spoiler.json.gz'

다운로드 된 압축된 gzip 풀어 loading하는 load_data() 함수를 작성합니다.

In [ ]:
def load_data(file_name):
    count = 0
    data = []
    with gzip.open(file_name) as fin:
        for l in fin:
            d = json.loads(l)
            count += 1
            data.append(d)
    return data

다운로드한 데이터를 review_data 변수에 로딩합니다.

In [ ]:
review_data = load_data(output) # 약 1분 가량 소요됨

Pandas DataFrame으로 저장한 후, 간단한 데이터셋 통계와 샘플을 출력해 봅니다. user_id, timestamp, review_sentences, rating, has_spoiler, book_id, review_id colum으로 이루어진 데이터셋임을 알 수 있습니다.

In [ ]:
df = pd.DataFrame(review_data)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378033 entries, 0 to 1378032
Data columns (total 7 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   user_id           1378033 non-null  object
 1   timestamp         1378033 non-null  object
 2   review_sentences  1378033 non-null  object
 3   rating            1378033 non-null  int64 
 4   has_spoiler       1378033 non-null  bool  
 5   book_id           1378033 non-null  object
 6   review_id         1378033 non-null  object
dtypes: bool(1), int64(1), object(5)
memory usage: 64.4+ MB


,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, This is a special book.], [0, It started ...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,"[[0, Recommended by Don Katz.], [0, Avail for ...",3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, A fun, fast paced science fiction thrille...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,"[[0, Recommended reading to understand what is...",0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, I really enjoyed this book, and there is ...",4,True,25884323,332732725863131279a8e345b63ac33e


## 2. Data Preprocessing
```
💡 목차 개요 : 이어서 Sequential Recommendation에 적합한 데이터로 전처리를 진행하겠습니다.
우리는 각 user가 시간대별로 어떤 책을 읽었는지 (평점을 남겼는지)에만 관심이 있으므로, user_id, timestamp, book_id 세 column만 따로 사용하겠습니다.
```

In [ ]:
df_review = df[['user_id', 'timestamp', 'book_id']]
df_review

,user_id,timestamp,book_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,18245960
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,16981
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,28684704
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,27161156
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,25884323
...,...,...,...
1378028,35cef391b171b4fca45771e508028212,2013-04-16,15745950
1378029,35cef391b171b4fca45771e508028212,2012-12-28,10861195
1378030,35cef391b171b4fca45771e508028212,2013-03-25,6131164
1378031,35cef391b171b4fca45771e508028212,2013-01-24,10025305


각 user 별 평점 갯수에 대한 통계를 내보겠습니다. 해당 통계를 통해, 무려 1815개의 평점을 남긴 user가 있는 반면, 1개 또는 소수의 평점만 작성한 user도 상당히 많은 것을 알 수 있습니다.

In [ ]:
df_review.groupby('user_id').size().value_counts().sort_index()

1       387
2       435
3       398
4       404
5       401
       ... 
1165      1
1189      1
1214      1
1647      1
1815      1
Length: 610, dtype: int64

데이터의 주요한 패턴에 집중하기 위해서 평점을 10개 이상 남긴 user만 가져오도록 하겠습니다.

In [ ]:
df_review = df_review.groupby('user_id').filter(lambda x: len(x) >= 10)

In [ ]:
df_review # 1378033 -> 1361333 으로 거의 차이 없음

,user_id,timestamp,book_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,18245960
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,16981
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,28684704
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,27161156
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,25884323
...,...,...,...
1378028,35cef391b171b4fca45771e508028212,2013-04-16,15745950
1378029,35cef391b171b4fca45771e508028212,2012-12-28,10861195
1378030,35cef391b171b4fca45771e508028212,2013-03-25,6131164
1378031,35cef391b171b4fca45771e508028212,2013-01-24,10025305


In [ ]:
book_ids = df_review['book_id'].unique()
user_ids = df_review['user_id'].unique()
num_item, num_user = len(book_ids), len(user_ids)

Filtering 후 간단한 통계를 내봅니다.
25475권의 책과 15451명의 사용자로 약 136만개의 rating 데이터가 구성되어 있음을 알 수 있습니다. 또한, 평균적으로 한명의 user가 약 88개의 평점을 작성했다는 정보도 알 수 있습니다.

In [ ]:
print ("unique book #: ", len(book_ids), "\nunique user #: ", len(user_ids), "\nrating #: ", len(df_review))
print ("mean rating #: ", len(df_review) / len(user_ids))

unique book #:  25475 
unique user #:  15451 
rating #:  1361333
mean rating #:  88.10646560093198


"8842281e1d1347389f2ab93d60773d4d"와 같이 복잡한 문자열 대신 깔끔한 정수로 book_id와 user_id를 reindexing하고, 새로 정의된 column명으로 구성된 df_review를 재구성합니다.

In [ ]:
book2idx = pd.Series(data=np.arange(len(book_ids))+1, index=book_ids) # item re-indexing (1~num_item) # 0 for padding in SASRec
user2idx = pd.Series(data=np.arange(len(user_ids)), index=user_ids) # user re-indexing (0~num_user-1)

df_review = pd.merge(df_review, pd.DataFrame({'book_id': book_ids, 'book_idx': book2idx[book_ids].values}), on='book_id', how='inner')
df_review = pd.merge(df_review, pd.DataFrame({'user_id': user_ids, 'user_idx': user2idx[user_ids].values}), on='user_id', how='inner')

df_review.sort_values(['user_idx', 'timestamp'], inplace=True) # user_idx와 timestamp로 정렬합니다.
df_review = df_review[['user_idx','timestamp', 'book_idx']]

In [ ]:
df_review

,user_idx,timestamp,book_idx
73,0,2006-12-07,74
55,0,2007-04-09,56
36,0,2009-04-14,37
35,0,2009-04-20,36
34,0,2009-06-18,35
...,...,...,...
1148408,15450,2013-12-25,1977
1148407,15450,2013-12-27,1364
1148405,15450,2014-02-03,939
1148416,15450,2014-02-03,5846


마지막으로 train 데이터와 test 데이터까지 구축하면 SASRec 모델을 위한 data preprocessing 작업은 마무리됩니다.

먼저 evaluation에 사용될 1000명의 유저 데이터를 미리 뽑아놓은 뒤, train-test split을 진행합니다. Sequential recommendaiotn 시나리오이기 때문에, 시간 순서로 정렬된 사용자의 아이템 시퀀스에서 마지막 아이템을 test 용도로, 나머지 아이템들을 train 용도로 사용합니다.

In [ ]:
num_item_sample = 100
num_user_sample = 1000

# 평가에 사용할 1000명의 사용자를 추출합니다.
users_for_eval = np.random.choice(range(num_user), num_user_sample, replace=False)

In [ ]:
eval_dict = {i: 0 for i in users_for_eval}
print (len(eval_dict))

1000


In [ ]:
# 먼저 데이터에 user split을 적용합니다. (train user와 test user가 서로 disjoint set)
# 가장 먼저 user별 시퀀스를 나누어서 저장하기 위한 defaultdict 객체들을 각각 선언합니다.
# df_reviews 데이터프레임을 순회하면서
# 각 user u가 eval_dict안에 포함되면, `users_eval`에 해당 시퀀스를 넣고
# 그렇지 않으면 `users`에 넣어줍니다.

# defaultdict은 dictionary의 key가 없을때 default 값을 value로 반환해 줍니다.
users = defaultdict(list) # for trainining
users_eval = defaultdict(list) # for evaluation

for u, i, t in zip(df_review['user_idx'], df_review['book_idx'], df_review['timestamp']):
    if u not in eval_dict:
        users[u].append(i)
    else:
        users_eval[u].append(i)

In [ ]:
# 학습셋과 평가셋의 사용자 별로 마지막 아이템을 test로, 나머지를 train으로 분리합니다.
user_train = {}
user_valid = {}
user_eval_train = {}
user_eval_test = {}

# 이때 이미 sequence는 시간순으로 정렬된 상태임에 유의해주세요.
for user in users:
    user_train[user] = users[user][:-1]
    user_valid[user] = [users[user][-1]]

for user in users_eval:
    user_eval_train[user] = users_eval[user][:-1]
    user_eval_test[user] = [users_eval[user][-1]]

training과 evaluation을 위한 user 갯수가 잘 맞는지 확인해 줍니다.

(# of user ids = # of user_train + # of user_eval_train)

In [ ]:
print(f"train user의 개수 {len(user_train)}와 test user 개수 {len(user_eval_train)}를 더하면 전체 user 수 {len(user_ids)}가 되어야 합니다.")

train user의 개수 14451와 test user 개수 1000를 더하면 전체 user 수 15451가 되어야 합니다.


User sequence가 잘 만들어졌는지 몇 개의 샘플을 통해 확인해봅니다.

In [ ]:
print (len(user_train), len(user_valid))
print (len(user_train[1]), len(user_valid[1]))
print (user_train[1])
print (user_valid[1])

print (len(user_train[5]), len(user_valid[5]))
print (user_train[5])
print (user_valid[5])

14451 14451
575 1
[658, 653, 652, 650, 649, 648, 647, 646, 644, 657, 643, 654, 642, 641, 645, 638, 635, 621, 622, 627, 611, 623, 631, 655, 632, 619, 620, 601, 617, 628, 633, 608, 626, 600, 613, 598, 637, 593, 597, 603, 599, 590, 624, 586, 587, 607, 615, 612, 640, 577, 604, 609, 576, 569, 570, 639, 567, 565, 580, 555, 575, 566, 594, 595, 636, 560, 564, 545, 544, 551, 538, 556, 563, 574, 579, 553, 549, 543, 534, 527, 524, 585, 602, 583, 537, 515, 605, 591, 511, 589, 539, 535, 548, 550, 532, 568, 504, 529, 518, 614, 498, 506, 494, 491, 513, 541, 490, 505, 530, 495, 486, 487, 514, 606, 501, 610, 484, 496, 517, 503, 499, 480, 492, 618, 478, 571, 477, 475, 489, 474, 540, 559, 471, 467, 596, 469, 465, 481, 523, 476, 508, 562, 581, 536, 519, 464, 502, 520, 616, 625, 531, 582, 500, 452, 473, 483, 448, 493, 466, 459, 546, 450, 552, 528, 445, 533, 482, 453, 436, 468, 561, 451, 479, 547, 440, 431, 457, 458, 572, 447, 430, 588, 455, 554, 419, 423, 463, 439, 443, 416, 435, 438, 420, 525, 427, 402, 4

## 3. SASRec Model
```
💡 목차 개요 : PyTorch로 SASRec 모델을 구현하고, 모델 학습 및 평가를 수행합니다.
```
* 3-1. Batch Sampling
* 3-2. 모델 구현
* 3-3. Model Training
* 3-4. Model Evaluation


### 3-1. Batch Sampling
> Negative sampling process을 포함하여 SASRec 모델에 사용될 batch 데이터를 생성합니다. Negative sampling은 모델이 사용자가 선호하지 않을 것으로 예상되는 item을 학습하는데 필요한 process로, 실습에서는 사용자가 읽지 않은 (rating을 남기지 않은) 책을 negativs sample로 간주하겠습니다.

In [ ]:
# for training, data sampling
def random_neg(l, r, s):
    # 데이터에 존재하는 아이템과 겹치지 않도록 sampling
    # l과 r 사이에서 무작위 정수를 선택합니다.
    t = np.random.randint(l, r)

    # 선택된 아이템이 사용자 sequence (s)에 이미 존재하는 경우 새로운 아이템 다시 샘플링
    # 즉, 샘플된 아이템이 user sequence (s)에 포함되지 않는 것이 나올 때까지 (rating을 남기지 않은 것일때까지) 샘플링 반복
    while t in s:
        t = np.random.randint(l, r)

    return t

In [ ]:
random_neg(1, 10, [2,4,6,8,10]) # 1부터 10까지의 정수 중 [2,4,6,8,10]을 제외한 홀수값만 샘플링됨

7

In [ ]:
# a = [1,2,3,4]
# for i in reversed(a[:-1]):
#   print(i)
'''
3
2
1
'''

'\n3\n2\n1\n'

In [ ]:

def sample_batch(user_train, num_user, num_item, batch_size, max_len):
    # 개별 사용자에 대한 데이터 샘플링
    def sample():
        # 앞서 생성한 eval_dict에 없는 사용자를 무작위로 샘플링
        while 1:
            user = np.random.randint(num_user)
            if user not in eval_dict:
                break

        # array 초기화
        seq = np.zeros([max_len], dtype=np.int32) # 유저별 아이템 시퀀스
        pos = np.zeros([max_len], dtype=np.int32) # positive ground-truth item
        neg = np.zeros([max_len], dtype=np.int32) # sampled negative item
        nxt = user_train[user][-1] # 해당 user sequence의 마지막 아이템을 nxt에 저장함
        idx = max_len - 1 # idx는 max_len에서 1뺀 값으로 사용함

        # 사용자의 item sequence를 역순으로 순회
        train_item = set(user_train[user])
        for i in reversed(user_train[user][:-1]): # 어차피 마지막 원소는 nxt에 지정되어 있으므로 빼고 순회함
            '''
            user_train[user]=[1,2,3,4], max_length=50 일때
              nxt = 4, idx = 49
              for문의 i는 3 -> 2 -> 1 (item_id) 순으로 순회함
              따라서 seq[49] = 3, pos[49] = 4, neg[49] = 네거티브 샘플링된 값
              이후, nxt에는 3가 할당되고, idx -= 1을 통해 idx = 48이 됨
              즉, 길이 50의 시퀀스에 대해 for문을 돌면서, input에 대응하는 output을 끝에서부터 채우는 형태임

            for문을 다 돌고 나면
              seq = [0,0,0,...,1,2,3]
              pos = [0,0,0,...,2,3,4]
              neg = [0,0,0,...,42,123,230]
            와 같이 채워지게 됨
            '''
            # 미리 정의된 sequence를 끝에서부터 역순으로 채움, ex: seq = [0,0,0,1,2,3] (0은 pad)
            seq[idx] = i #
            pos[idx] = nxt # 위에서 지정한 마지막 아이템이 positive item으로 지정됨
            if nxt != 0:
                neg[idx] = random_neg(1, num_item + 1, train_item) # negative sampling
            nxt = i # 현재의 i를 (nxt 이전 값)을 다음의 nxt로 사용하도록 변경
            idx -= 1 # idx도 한 스텝 진행시킴 (역순이므로 -1)
            if idx == -1: break
        return (user, seq, pos, neg)

    # 배치 사이즈만큼 데이터 샘플링
    user, seq, pos, neg = zip(*[sample() for _ in range(batch_size)])
    user, seq, pos, neg = np.array(user), np.array(seq), np.array(pos), np.array(neg)
    return user, seq, pos, neg

> sample_batch 함수는 일견 복잡해보이지만, 간단한 샘플 입력에 대해 각 변수들을 print()해보면 직관적으로 이해할 수 있습니다. seq라는 input에 pos라는 output이 한 스텝씩 shifted 되어서 입력되는 것은 다음의 SASRec 그림에서도 알 수 있습니다.


<div align="center">
<img src="https://recbole.io/docs/_images/sasrec.png" width="60%">
<figcaption>SASRec Architecture</figcaption>
</div>

#### 📚 자료:
* [Self-Attentive Sequential Recommendation](https://arxiv.org/pdf/1808.09781.pdf)

### 3-2. 모델 구현
> Transformers를 이루는 ScaledDotProductAttention, MultiHeadAttention 및 PositionwiseFeedForward 클래스를 구현하는 부분은 워낙 널리 알려져 있는 부분이기 때문에 생략하고 넘어가도록 하겠습니다. 특히 MultiHeadAttention은 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html)과 같이 PyTorch 내부 함수로도 지원하기도 합니다. 이후에는 SASRec 블록 위주로 살펴보겠습니다.

* Scaled-Dot Product Attention 구현

In [ ]:
class ScaledDotProductAttention(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(ScaledDotProductAttention, self).__init__()
        self.hidden_units = hidden_units
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, Q, K, V, mask):
        # ScaledDotProductAttention 점수 계산
        attn_score = torch.matmul(Q, K.transpose(2, 3)) / math.sqrt(self.hidden_units)

        # 마스킹된 위치의 score을 -1e9로 설정하여 softmax 통과 시 0이 되도록 합니다
        attn_score = attn_score.masked_fill(mask == 0, -1e9)

        # Attention distribution 계산 및 dropout 설정
        attn_dist = self.dropout(F.softmax(attn_score, dim=-1))

        # Attention 가중치와 value (V) 곱 계산
        # dim of output : batchSize x num_head x seqLen x hidden_units
        output = torch.matmul(attn_dist, V)

        return output, attn_dist

* Multi-head Attention 구현

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads # head 갯수
        self.hidden_units = hidden_units

        # linear transformation을 위한 가중치 정의
        self.W_Q = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_K = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_V = nn.Linear(hidden_units, hidden_units, bias=False)
        self.W_O = nn.Linear(hidden_units, hidden_units, bias=False)

        # 앞서 정의한 ScaledDotProductAttention class를 통한 Attention layer 초기화
        self.attention = ScaledDotProductAttention(hidden_units, dropout_rate)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6)

    def forward(self, enc, mask):
        residual = enc # residual connection을 위한 입력 저장
        batch_size, seqlen = enc.size(0), enc.size(1)

        # Query, Key, Value 계산
        # num_head개의 head로 나누어 각기 다른 Linear projection을 통과시킵니다
        Q = self.W_Q(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        K = self.W_K(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)
        V = self.W_V(enc).view(batch_size, seqlen, self.num_heads, self.hidden_units)

        # multi-head attantion을 위한 transpose
        Q, K, V = Q.transpose(1, 2), K.transpose(1, 2), V.transpose(1, 2)
        output, attn_dist = self.attention(Q, K, V, mask)

        # attention 결과 병합 및 최종 출력 계산
        output = output.transpose(1, 2).contiguous().view(batch_size, seqlen, -1)
        output = self.layerNorm(self.dropout(self.W_O(output)) + residual)
        return output, attn_dist

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, hidden_units, dropout_rate):
        super(PositionwiseFeedForward, self).__init__()

        self.W_1 = nn.Linear(hidden_units, hidden_units)
        self.W_2 = nn.Linear(hidden_units, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.layerNorm = nn.LayerNorm(hidden_units, 1e-6)

    def forward(self, x):
        # residual connextion을 위해 입력 미리 저장
        residual = x
        output = self.W_2(F.relu(self.dropout(self.W_1(x))))
        output = self.layerNorm(self.dropout(output) + residual)
        return output

* SASRec Block 및 SASRec Class 구현:
다음으로 SASRec의 주요 구성 요소인 SASRec Block을 구현하겠습니다. SASRec Block은 multi-head attention와 위치 별 feedforward network를 포함합니다.

In [ ]:
class SASRecBlock(nn.Module):
    # 레이어 초기화
    def __init__(self, num_heads, hidden_units, dropout_rate):
        super(SASRecBlock, self).__init__()
        self.attention = MultiHeadAttention(num_heads, hidden_units, dropout_rate)
        self.pointwise_feedforward = PositionwiseFeedForward(hidden_units, dropout_rate)

    def forward(self, input_enc, mask):
        # multi-head attention 레이어 통과
        output_enc, attn_dist = self.attention(input_enc, mask)
        output_enc = self.pointwise_feedforward(output_enc)
        return output_enc, attn_dist

In [ ]:
# SASRec은 위에서 구현한 SASRecBlock을 여러 층으로 쌓고 item_emb 및 pos_emb 등을 준비합니다.
class SASRec(nn.Module):
    def __init__(self, num_user, num_item, hidden_units, num_heads, num_layers, maxlen, dropout_rate, device):
        super(SASRec, self).__init__()

        self.num_user = num_user
        self.num_item = num_item
        self.hidden_units = hidden_units
        self.num_heads = num_heads
        self.num_layers = num_layers
        self.device = device
        # padding index를 포함한 item embedding 레이어
        self.item_emb = nn.Embedding(num_item + 1, hidden_units, padding_idx=0)
        # learnable postiional encoding -> position_embedding
        self.pos_emb = nn.Embedding(maxlen, hidden_units)
        self.dropout = nn.Dropout(dropout_rate)
        self.emb_layernorm = nn.LayerNorm(hidden_units, eps=1e-6)

        # SASRec 블록 (transformer 블록)을 여러 층으로 쌓습니다
        self.blocks = nn.ModuleList([SASRecBlock(num_heads, hidden_units, dropout_rate) for _ in range(num_layers)])

    # sequence의 특성 (item embedding, positional embedding) 추출 함수
    # sequence를 item_emb을 통과시켜서 sequence embedding을 얻은 뒤, 이를 각 포지션 index에 대응하는 pos_emb을 더해줍니다.
    # 이후 Dropout 및 LayerNorm을 적용합니다.
    def feats(self, log_seqs):
        seqs = self.item_emb(torch.LongTensor(log_seqs).to(self.device))
        positions = np.tile(np.array(range(log_seqs.shape[1])), [log_seqs.shape[0], 1])
        seqs += self.pos_emb(torch.LongTensor(positions).to(self.device))
        seqs = self.emb_layernorm(self.dropout(seqs))

        # 마스킹: padding된 부분을 마스킹합니다
        # unsqueeze()를 통해 mask_pad와 mask_time의 차원을 맞춰줍니다
        mask_pad = torch.BoolTensor(log_seqs > 0).unsqueeze(1).unsqueeze(1) # log_seqs=0인 경우 masking이 필요합니다. 해당 조건을 만족하는 mask를 구현
        # sequence 순서를 고려한 마스킹 구현 (현재 시점에서 미래 시점의 정보를 보고 예측하는 leakage를 피하기 위함)
        # 해당 mask는 causal attention mask라고도 하며, Transformer decoder에 사용됩니다.
        mask_time = (1 - torch.triu(torch.ones((1, 1, seqs.size(1), seqs.size(1))), diagonal=1)).bool()
        mask = (mask_pad & mask_time).to(self.device)
        for block in self.blocks:
            seqs, attn_dist = block(seqs, mask)
        return seqs

    # training에 사용되는 함수
    def forward(self, log_seqs, pos_seqs, neg_seqs):
        feats = self.feats(log_seqs)
        pos_embs = self.item_emb(torch.LongTensor(pos_seqs).to(self.device))
        neg_embs = self.item_emb(torch.LongTensor(neg_seqs).to(self.device))

        pos_logits = (feats * pos_embs).sum(dim=-1)
        neg_logits = (feats * neg_embs).sum(dim=-1)
        return pos_logits, neg_logits

    # evaluation에 사용되는 함수
    def predict(self, log_seqs, item_indices):
        final_feats = self.feats(log_seqs)[:, -1, :]
        item_embs = self.item_emb(torch.LongTensor(item_indices).to(self.device))
        logits = item_embs.matmul(final_feats.unsqueeze(-1)).squeeze(-1)
        return logits

In [ ]:
  example_seq = torch.tensor([[1,2,3,4]])
  mask_pad = torch.BoolTensor(example_seq > 0).unsqueeze(1).unsqueeze(1)
  mask_time = (1 - torch.triu(torch.ones((1, 1, example_seq.size(1), example_seq.size(1))), diagonal=1)).bool()
  mask = (mask_pad & mask_time)
  mask

tensor([[[[ True, False, False, False],
          [ True,  True, False, False],
          [ True,  True,  True, False],
          [ True,  True,  True,  True]]]])

In [ ]:
'''
ScaledDotProductAttention 클래스 구현 중...

    # 마스킹된 위치의 score을 -1e9로 설정하여 softmax 통과 시 0이 되도록 합니다
    attn_score = attn_score.masked_fill(mask == 0, -1e9)
    ...

    # Query와 Key의 행렬곱 연산 이후 이전 시점에 대해 이후 시점의 영향력을 배제하기
    # 위한 Attention Masking을 수행합니다. 이는 이후 시점의 item에 해당하는 값들을
    # 모두 0으로 마스킹함으로써 Value와의 행렬곱 연산에서 이후 시점들의 값을 반영하지 않도록 해줍니다.
'''
attn_score = torch.randn_like(mask.float())
attn_score.masked_fill(mask==0, -1e9)

tensor([[[[ 7.6532e-01, -1.0000e+09, -1.0000e+09, -1.0000e+09],
          [-8.3982e-01,  8.7288e-01, -1.0000e+09, -1.0000e+09],
          [-7.5211e-01,  5.4127e-01, -2.1073e-01, -1.0000e+09],
          [-1.9752e-02, -4.4853e-01, -1.3245e+00,  7.3228e-01]]]])

> 아래의 직관적인 예시 그림을 통해 현재 시점 이후의 token들은 모두 mask를 써서 다음 값의 예측에 사용되지 않도록 할 수 있음을 알 수 있습니다.


  <img src="https://velog.velcdn.com/images%2Fseven7724%2Fpost%2Ff1f1daf0-9723-4c32-b96e-6a4664bbcb7e%2F%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-11-08%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%207.00.25.png">

  <img src="https://velog.velcdn.com/images%2Fseven7724%2Fpost%2F9d39b43e-f342-487c-9ae6-288becb84e79%2F%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-11-08%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%207.00.34.png">

  <img src="https://velog.velcdn.com/images%2Fseven7724%2Fpost%2F9e9d96db-4ebb-4caa-a82b-3d5aa9d63a76%2F%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202021-11-08%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%207.00.41.png">

#### 📚 자료:
* [Transformer 관련 블로그 리뷰](https://velog.io/@seven7724/TransformerAttention-Is-All-You-Need)



### 3-3. Model Training
>

* 학습에 필요한 하이퍼파라미터 세팅

In [ ]:
# hyperparameter setting
max_len = 50
hidden_units = 50
num_heads = 1
num_layers = 2
dropout_rate=0.5
num_workers = 1
device = 'cuda'

# training setting
lr = 1e-3
num_epochs = 10

batch_size = 128
num_batch = num_user // batch_size

* 손실 함수 및 옵티마이저 설정

In [ ]:
# model instance 생성
model = SASRec(num_user, num_item, hidden_units, num_heads, num_layers, max_len, dropout_rate, device)
model.to(device)

# 손실 함수 정의: Binary Cross-Entropy Loss를 통해 실제 상호작용과 예측값 간의 차이를 계산합니다
criterion = torch.nn.BCEWithLogitsLoss()

# 최적화 함수 정의
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

* 모델 학습:
모델을 학습하고, 손실이 잘 떨어지는지 관찰합니다. 필요에 따라, epoch 등을 조절합니다.

In [ ]:
# Model training loop
# 각 epoch에서 num_batch만큼의 배치를 처리합니다
for epoch in range(1, num_epochs + 1):
    tbar = tqdm(range(num_batch))
    for step in tbar:
        # sample_batch 함수를 통해 사용자, 아이템 sequence, postive 및 negative 샘플을 받습니다
        user, seq, pos, neg = sample_batch(user_train, num_user, num_item, batch_size, max_len)

        # positive 및 negative sample에 대한 logit 출력
        pos_logits, neg_logits = model(seq, pos, neg)
        # positive는 1, negavite는 0으로 레이블링
        pos_labels, neg_labels = torch.ones(pos_logits.shape, device=device), torch.zeros(neg_logits.shape, device=device)

        optimizer.zero_grad()
        # padding되지 않은 부분에 대해서만 손실 계산
        indices = np.where(pos != 0)
        # BCE를 통한 손실 계산
        loss = criterion(pos_logits[indices], pos_labels[indices])
        loss += criterion(neg_logits[indices], neg_labels[indices])

        loss.backward()
        optimizer.step()

        tbar.set_description(f'Epoch: {epoch:3d}| Step: {step:3d}| Train loss: {loss:.5f}')

Epoch:   1| Step: 119| Train loss: 4.29079: 100%|██████████| 120/120 [00:04<00:00, 25.69it/s]
Epoch:   2| Step: 119| Train loss: 2.97307: 100%|██████████| 120/120 [00:04<00:00, 28.82it/s]
Epoch:   3| Step: 119| Train loss: 2.12340: 100%|██████████| 120/120 [00:05<00:00, 23.52it/s]
Epoch:   4| Step: 119| Train loss: 1.68599: 100%|██████████| 120/120 [00:04<00:00, 27.53it/s]
Epoch:   5| Step: 119| Train loss: 1.47507: 100%|██████████| 120/120 [00:04<00:00, 28.96it/s]
Epoch:   6| Step: 119| Train loss: 1.43633: 100%|██████████| 120/120 [00:05<00:00, 22.93it/s]
Epoch:   7| Step: 119| Train loss: 1.38005: 100%|██████████| 120/120 [00:04<00:00, 28.67it/s]
Epoch:   8| Step: 119| Train loss: 1.35354: 100%|██████████| 120/120 [00:04<00:00, 28.55it/s]
Epoch:   9| Step: 119| Train loss: 1.31002: 100%|██████████| 120/120 [00:05<00:00, 22.90it/s]
Epoch:  10| Step: 119| Train loss: 1.28529: 100%|██████████| 120/120 [00:04<00:00, 28.83it/s]


### 3-4. Model Evaluation
> 마지막으로, 앞전에 미리 뽑아둔 평가를 위한 1000개의 user data를 사용하여 학습된 모델을 평가하겠습니다. 평가 지표로는 랭킹 기반인 NDCG와 HIT을 사용합니다.

> Hit Rate (HR)@K 또는 Hit@K는 전체 사용자 수 대비 적중한 사용자의 수를 의미합니다 (적중률). 보통 사용자 별로 ground-truth 아이템을 1개만 남겨둔 Leave One Last 분할 전략에서 주로 사용됩니다 (아이템 1개를 맞췄는지 못 맞췄는지는 명확하므로). 즉, 사용자 별로 K개의 아이템을 추천했을 때, ground-truth 아이템을 포함하고 있다면 `hit=1`, 아니면 `hit=0`을 할당하고, 해당 `hit` 값을 사용자 별로 평균냄으로써 Hit@K를 얻을 수 있습니다.

In [ ]:
num_item_sample = 100
model.eval()

NDCG = 0.0 # NDCG@10
HIT = 0.0 # Hit Rate@10

for u in users_eval:
    seq = user_eval_train[u][-max_len:]
    rated = set(user_eval_train[u] + user_eval_test[u])
    # uniform-sampled-based ranking evaluation을 위해 사용자 별로 100개의 negative item을 샘플합니다.
    # 이 부분을 학습에 사용되는 부분과 헷갈리시는 분들이 계십니다. 동일한 로직이지만 사용 목적이 미묘하게 다릅니다.
    # 학습 시에는 매 번 positive item에 대응하는 contrastive loss를 만들기 위해서 사용됩니다.
    # 평가 시에는 모든 item에 대한 scoring-ranking이 오래 걸리기 때문에, 빠른 평가를 위해서 사용자 별로 negative item을 미리 샘플해둔 채로 사용합니다.
    # 그리고 미리 준비한 유저 별 1개의 ground-truth 아이템 이를 합친 뒤에 평가 메트릭을 계산합니다.
    # (e.g., candidate_items = one_gt_item + sampled_negative_items)
    item_idx = user_eval_test[u] + [random_neg(1, num_item + 1, rated) for _ in range(num_item_sample)]

    with torch.no_grad():
        predictions = -model.predict(np.array([seq]), np.array(item_idx))
        predictions = predictions[0]
        rank = predictions.argsort().argsort()[0].item()

    # 평가할 아이템이 상위 10개에 있는지 확인
    if rank < 10: # at 10
        NDCG += 1 / np.log2(rank + 2)
        HIT += 1

# 최종 성능 출력
print(f'NDCG@10: {NDCG/num_user_sample}| HR@10: {HIT/num_user_sample}')

NDCG@10: 0.19871610420844518| HR@10: 0.344


## 4. 마치며
이번 실습에서는 먼저 sequential data를 처리하는 기법에 대해 이해해보고, SASRec 모델을 실제 GoodReads 데이터에 적용하여 사용자의 이전 상호작용을 기반으로 한 next item prediction을 직접 구현해 보았습니다. Vanilla transformers를 기반으로 하는 비교적 간단한 sequential recommendation 모델인 SASRec 모델 구현 실습 경험이 앞으로 보다 복잡한 sequential recommendation 모델들을 구현하는 데 기초가 되기를 바랍니다.


## References

- [Self-Attentive Sequential Recommendation](https://arxiv.org/pdf/1808.09781.pdf)
- [Attnetion Is All You Need](https://arxiv.org/abs/1706.03762)
- [SASRec Official Implementation](https://github.com/kang205/SASRec)
- [PyTorch 기반의 SASRec 구현](https://github.com/pmixer/SASRec.pytorch)
- [Transformer 관련 블로그 리뷰](https://velog.io/@seven7724/TransformerAttention-Is-All-You-Need)

## Required Package

- tqdm==4.66.1
- numpy==1.23.5
- torch==2.1.0+cu121
- pandas==1.5.3

## 콘텐츠 라이선스

저작권 : <font color='blue'> <b> ©2023 by Upstage X fastcampus Co., Ltd. All rights reserved.</font></b>

<font color='red'><b>WARNING</font> : 본 교육 콘텐츠의 지식재산권은 업스테이지 및 패스트캠퍼스에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. </b>